<a href="https://colab.research.google.com/github/chinnakritsf-source/Traffic_prediction/blob/main/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from datetime import datetime

print("NYC Traffic Speed Prediction - Fast + Accurate\n")

# Fetch optimal amount of data (5000 records - balance speed/accuracy)
print("Fetching data...")
url = 'https://data.cityofnewyork.us/resource/i4gi-tjb9.json?$limit=5000'
df = pd.DataFrame(requests.get(url).json())
print(f"✓ Loaded {len(df)} records\n")

# Clean data
df['speed'] = pd.to_numeric(df['speed'], errors='coerce')
df['data_as_of'] = pd.to_datetime(df['data_as_of'])
df['hour'] = df['data_as_of'].dt.hour
df['day_of_week'] = df['data_as_of'].dt.dayofweek
df['link_id'] = pd.to_numeric(df['link_id'], errors='coerce')
df = df.dropna(subset=['speed', 'hour', 'day_of_week', 'link_id'])

# Create features (keep important ones only)
df_grouped = df.groupby(['link_id', 'hour', 'day_of_week']).agg({'speed': 'mean'}).reset_index()
df_grouped['speed_lag_1h'] = df_grouped.groupby('link_id')['speed'].shift(1)
df_grouped = df_grouped.dropna()

if len(df_grouped) > 50:
    X = df_grouped[['hour', 'day_of_week', 'speed_lag_1h']]
    y = df_grouped['speed']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Use Gradient Boosting (faster than RF, similar accuracy)
    print("Training model...")
    model = GradientBoostingRegressor(
        n_estimators=50,        # Reduced from 100
        max_depth=5,            # Limit tree depth
        learning_rate=0.1,
        random_state=42
    )
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    accuracy = (1 - mae/y_test.mean()) * 100

    # Current traffic analysis
    current_hour = datetime.now().hour
    current_speeds = df[df['hour'] == current_hour]['speed']
    total_segments = len(current_speeds)
    avg_speed = current_speeds.mean() if total_segments > 0 else df['speed'].mean()
    slow_traffic = len(current_speeds[current_speeds < 15]) if total_segments > 0 else 0
    pct_slow = (slow_traffic/total_segments)*100 if total_segments > 0 else 0

    print(f"\n{'='*50}")
    print("LIVE TRAFFIC CONDITIONS")
    print(f"{'='*50}")
    print(f"Current Time: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
    print(f"Active Road Segments: {total_segments}")
    print(f"Average Speed: {avg_speed:.1f} mph")
    print(f"Slow Traffic Segments: {slow_traffic} ({pct_slow:.1f}%)")
    print(f"\nModel Accuracy: {accuracy:.1f}%")
    print(f"Prediction Error: ±{mae:.2f} mph")

    if avg_speed < 15:
        print("\n🔴 HEAVY TRAFFIC - Expect delays!")
    elif avg_speed < 25:
        print("\n🟡 MODERATE TRAFFIC - Some congestion")
    else:
        print("\n🟢 LIGHT TRAFFIC - Good conditions")
else:
    print("⚠️ Not enough data available")

NYC Traffic Speed Prediction - Fast + Accurate

Fetching data...
✓ Loaded 5000 records

Training model...

LIVE TRAFFIC CONDITIONS
Current Time: 2026-02-01 16:02
Active Road Segments: 0
Average Speed: 35.4 mph
Slow Traffic Segments: 0 (0.0%)

Model Accuracy: 92.0%
Prediction Error: ±2.80 mph

🟢 LIGHT TRAFFIC - Good conditions
